Lesson9 Assignment

In [ ]:
# %load main.py
#Loading modules
from osgeo import ogr, osr
import os
#Changing working directory
os.chdir(os.path.dirname(__file__))

# Create driver
driverName = "ESRI Shapefile"
drv = ogr.GetDriverByName( driverName )

# Define path and names
a_file = 'data/points.shp'
layername = 'a_layer'
# Create shapefile
ds = drv.CreateDataSource(a_file)
# Give geo reference
spatialReference = osr.SpatialReference()
spatialReference.ImportFromEPSG(4326)

# Create layer
layer = ds.CreateLayer(layername, spatialReference, ogr.wkbPoint)
# Export to a kml file
kml_file = ogr.Open('data/points.kml')
for lay in kml_file:
    for feat in lay:
        geom = feat.GetGeometryRef()
        if geom != None:
            point = []
            for i in range(0, geom.GetPointCount()):
                coords = geom.GetPoint(i)
                point = ogr.Geometry(ogr.wkbPoint)
                point.SetPoint(0,coords[0],coords[1])
                layerDef = layer.GetLayerDefn()
                feature = ogr.Feature(layerDef)
                feature.SetGeometry(point)
                layer.CreateFeature(feature)

ds.Destroy()


In [ ]:
# %load map_view.py
# Module pyshp needed

import folium
import os

# From @frankrowe/shp2gj.py
import shapefile
# read the shapefile
reader = shapefile.Reader("data/points.shp")
fields = reader.fields[1:]
field_names = [field[0] for field in fields]
buffer = []
for sr in reader.shapeRecords():
    atr = dict(zip(field_names, sr.record))
    geom = sr.shape.__geo_interface__
    buffer.append(dict(type="Feature",geometry=geom, properties=atr)) 
   
# write the GeoJSON file
from json import dumps
geojson = open("data/points.json", "w")
geojson.write(dumps({"type": "FeatureCollection","features": buffer}, indent=2) + "\n")
geojson.close()
   
pointsGeo = os.path.join("data/points.json")
map_points = folium.Map(location=[51.97,5.66],tiles='Stamen Terrain', zoom_start=14)
map_points.choropleth(geo_path=pointsGeo)
map_points.save('data/points.html')

In [7]:
from IPython.display import HTML
HTML(filename='data/points.html')